In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the 'License');
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an 'AS IS' BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
from google.cloud import storage

# Replace with your Google Cloud project ID
project_id = "CHANGE-ME"  #####<CHANGE_ME>

location = "us-central1"

# Replace with your Google Cloud Storage bucket name
bucket_name = "CHANGE-ME"                 #####<CHANGE_ME>. E.g. "mybucket"

# Gemini Model to use
model_id="CHANGE-ME"     ##### E.g. "gemini-1.5-pro-001"

# BQ Table as output
table_id='CHANGE-ME.CHANGE-ME' #####<CHANGE_ME> E.g. 'mydataset.image_depth'

# CSV Input File
csv_file = "CHANGE-ME"   ## Local CSV file. E.g. "/home/user/sample_data_100.csv"


In [3]:
import requests
import base64

def download_image_as_base64(url):
  """
  Downloads an image from a given URL and returns the image data as a base64-encoded string.

  Args:
    url: The URL of the image.

  Returns:
    The base64-encoded image data as a string, or None if the download fails.
  """

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    image_bytes = response.content
    base64_encoded_image = base64.b64encode(image_bytes).decode('utf-8')  # Encode to base64 and decode to string
    return base64_encoded_image
  except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
    return None

In [4]:
text1 = """tell me the picture quality, if it\'s easy to read the numbers in the picture, raw_depth and explain why."""

# A valid Depth read contains one or more of these symbols or words close to it or next to it: feet, inches, ft, in, ftin, ft/in, \', \", ft:in.
'''
text2 = """
Valid depth units: feet, inches, ft, in, ftin, ft/in,',\", ft:in. ft/m is not a valid depth unit. 
Valid depth examples: 5:00ft:in, 2.9', 2'03\", 204FTIN, 033\", 23in, 202FT, 2.0ft, 15'11\", 4'6\"ft/in, 8.22ft,4:00ft:in, 4.11ft./in. 
Depth is not expressed in %, nor psi, not temperature, nor degrees.
2.6', for example, is not the same as 2'6\". 102" is not the same 1'02\". Therefore, do not try to convert the depth and return the original value.
Do not convert the depth. If the value includes ft:in, reply using the term ft:in.  
Do not replace FTIN with symbols ' and/or \". If response contains FTIN, reply using FTIN.
Do not add extra punctuation. 
Respond with the raw depth value.
If Ditch Witch, depth is the whole text on top right. Some DitchWitch have in, ft or none. Do not add in or ft if the reading does not say it. 
If DigTrak, depth is the top value in the middle. 
If 3 of less depth readings, choose the one on top right.
Only when more than 5 depth reads found, return the word Aggregate.
"""
'''

text2 = """
Respond with the raw depth value.
"""

text3 = """JSON Schema:
{ \"picture_quality\": \"\", \"quality_details\": \"\", \"easy_read\": \"\", \"raw_depth\":\"\", \"reason\": \"\" }

Example 1:"""
image1 = Part.from_data(
    mime_type="image/jpeg",
    data=base64.b64decode("""..."""))
text4 = """Output:
{ \"picture_quality\": \"so_so\", \"quality_details\": \"blurry\", \"easy_read\": \"yes\", \"raw_depth\":\"4\'8\\\"\", \"reason\": \"The image is slightly blurry, but the numbers are still easy to read.\" }

Example2:"""
image2 = Part.from_data(
    mime_type="image/jpeg",
    data=base64.b64decode("""..."""))
text5 = """Output:
{ \"picture_quality\": \"great\", \"quality_details\": \"clear\", \"easy_read\": \"yes\", \"raw_depth\":\"208FTIN\", \"reason\": \"The image is clear and the numbers are easy to read.\" }

Example3:"""
image3 = Part.from_data(
    mime_type="image/jpeg",
    data=base64.b64decode("""..."""))
text6 = """Output:
{ \"picture_quality\": \"great\", \"quality_details\": \"clear\", \"easy_read\": \"yes\", \"raw_depth\":\"69in\", \"reason\": \"The image is clear and the numbers are easy to read.\" }

Example4:"""
example4 = Part.from_data(
    mime_type="image/jpeg",
    data=base64.b64decode("""..."""))
text7 = """Output:
{ \"picture_quality\": \"great\", \"quality_details\": \"clear\", \"easy_read\": \"yes\", \"raw_depth\":\"3'02\\\"\", \"reason\": \"The image is clear and the numbers are easy to read.\" }

Example5:"""
example5 = Part.from_data(
    mime_type="image/jpeg",
    data=base64.b64decode("""..."""))
text8 = """Output:
{ \"picture_quality\": \"great\", \"quality_details\": \"clear\", \"easy_read\": \"yes\", \"raw_depth\":\"54in\", \"reason\": \"The image is clear and the numbers are easy to read.\" }

Example6:"""
example6 = Part.from_data(
    mime_type="image/jpeg",
    data=base64.b64decode("""..."""))
text9 = """Output:
{ \"picture_quality\": \"great\", \"quality_details\": \"clear\", \"easy_read\": \"yes\", \"raw_depth\":\"1'11\\\"\", \"reason\": \"The image is clear and the numbers are easy to read.\" }


Picture:"""
image4 = Part.from_data(
    mime_type="image/jpeg",
    data=base64.b64decode("""..."""))


generation_config = {
    "max_output_tokens": 1024,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}


In [5]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

def generate(img_bytes):
  vertexai.init(project=project_id, location=location)

  #model = GenerativeModel(
  #  "gemini-1.5-pro-001",
  #)

  model = GenerativeModel(
    model_id,
    system_instruction=[
        "You are a master depth reader.",
        "Your mission is to precisely identify the depth listed in the pictures.",
        "Make sure you are reading it correctly. Double check your response prior to returning the results.",
        f"""
        Valid depth units: feet, inches, ft, in, ftin, ft/in,',\", ft:in. ft/m is not a valid depth unit. 
        Valid depth examples: 5:00ft:in, 2.9', 2'03\", 204FTIN, 033\", 23in, 202FT, 2.0ft, 15'11\", 4'6\"ft/in, 8.22ft,4:00ft:in, 4.11ft./in. 
        2.6', for example, is not the same as 2'6\". 102" is not the same 1'02\". Therefore, do not try to convert the depth and return the original value.
        Depth is not expressed by %, psi, degrees, ft/m or temperature.
        Do not convert the depth. If the value includes ft:in, reply using the term ft:in.  
        Do not replace FTIN with symbols ' and/or \". If response contains FTIN, reply using FTIN.
        Do not add extra punctuation. 
        If Ditch Witch, depth is the whole text on top right. Some DitchWitch have in, ft or none. Do not add in or ft if the reading does not say it. 
        If DigTrak, depth is the top value in the middle. 
        If 3 of less depth readings, choose the one on top right.
        When more than 5 depth reads found, return the word Aggregate.
        When the word Obstacles is present, return the word Aggregate.""",
    ],
  )

  responses = model.generate_content(
      [text1, text2, """Respond in valid JSON format with attributes. Escape any special characters like double quote. Do not return markdown or ``` """, 
      """picture_quality - allowed values great, so_so, bad""", 
      """quality_details - allowed clear, blurry, illegible""", 
      """easy_read - allowed values yes or no""", 
      """reason - detailed explanation""", 
      """raw_depth - Raw depth value as shown on screen""", 
      text3, image1, text4, image2, text5, image3, text6, example4, text7, example5, text8, example6, text9, img_bytes, """Output:"""],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )

  
  #print(responses.text, end="")

  return responses.text


In [6]:
def clean_response(text):
  import re
  """Removes ```json from the beginning and ``` from the end of a string.

  Args:
    text: The string to clean.

  Returns:
    The cleaned string.
  """
  if text.startswith("```json"):
    text = text[7:]  # Remove "```json" from the beginning
  if text.endswith("```"):
    text = text[:-3]  # Remove "```" from the end
  if text.endswith("```"):
    text = text[:-3]  # Remove "```" from the end
  
  text = re.sub(r"```", "", text)
  
  return text




In [7]:
# Example usage
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/97934565?token=923C5E43-9B30-4B1C-BDDA-99D48C7DD674"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/93463720?token=464C81E3-1E39-4AF0-A672-EFB789C6DA73"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/97828498?token=DAAF1A54-DBD6-49D3-BE27-AF03623AECEF"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/97307069?token=5E03C7A8-901F-4956-9483-39C29004F920"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/83799390?token=E7BBB31D-EEFA-4373-B0A9-D1FC4740CA93"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/109046388?token=6BD92578-7C4A-409F-8F69-E4A24EE65789"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/90076363?token=891D3879-8763-40AB-B268-E32D9465A460"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/97307069?token=5E03C7A8-901F-4956-9483-39C29004F920"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/107054761?token=DB566341-9247-46D1-AA48-BB85BE10E07F"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/89285683?token=66A86E3F-800A-43BA-84D7-B727CBF422BF"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/90575591?token=F29C4002-4CA3-437A-BF28-BB1F7908F2FB"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/123188440?token=12164EEC-ECE5-4C21-B48D-C543A58EE57E"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/83799390?token=E7BBB31D-EEFA-4373-B0A9-D1FC4740CA93"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/97307069?token=5E03C7A8-901F-4956-9483-39C29004F920"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/97707244?token=3B4D39D4-9FA9-4A82-9B3A-968D2FC95535"
#image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/111674526?token=195FA06F-5ABF-4BCE-8F35-3988B62DC8AF"
#image_url = 'https://fieldaudit.certusview.com/service/Audit/Asset/107054772?token=DB566341-9247-46D1-AA48-BB85BE10E07F'
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/107054775?token=DB566341-9247-46D1-AA48-BB85BE10E07F"
image_url = "https://fieldaudit.certusview.com/service/Audit/Asset/93920990?token=1C984F53-F886-4811-B9A1-D4F3F8F49D6B"
base64_image = download_image_as_base64(image_url)
image4 = Part.from_data(mime_type="image/jpeg",data=base64.b64decode(base64_image))
#image4 = base64_image
response = clean_response(generate(image4))
print ("Gemini response: " + response)

Gemini response: {
 "picture_quality": "great",
 "quality_details": "clear",
 "easy_read": "yes",
 "raw_depth":"2'08\"",
 "reason": "The image is clear and the numbers are easy to read."
}



In [8]:
import pandas as pd

def get_most_frequent_depth(response_df):
  """
  This function takes a Pandas DataFrame with columns 'depth' and 'full_response'
  and returns the row with the most occurrences of the 'depth' value.

  If there is a tie on depth count, the first row with the highest count is returned.
  """

  depth_counts = response_df['depth'].value_counts()
  most_frequent_depth = depth_counts.idxmax()
  
  # Get the first row with the most frequent depth as a DataFrame
  if len(response_df[response_df['depth'] == most_frequent_depth]) > 0:
    return response_df[response_df['depth'] == most_frequent_depth].iloc[[0]]
  else:
    return response_df.iloc[[0]] # Return the first row of the DataFrame



In [30]:
def modify_depth(depth: str) -> str:
  import re
  
  """
  Modifies the input string based on specific conditions.

  Args:
    depth: The input string.

  Returns:
    The modified string.
  """

  # Remove punctuation if FTIN is present (case-insensitive)
  if 'FTIN' in depth.upper():
    depth = depth.replace('.', '')
    depth = depth.replace(',', '')
    depth = depth.replace("'", '')
    depth = depth.replace('"', '')

  # Replace "." with ":" if "FT:IN" is present (case-insensitive)
  if "FT:IN" in depth.upper() or "FT:IN" in depth.lower():
        depth = depth.replace(".", "")
        depth = depth.replace("'", "")

  # Replace "." with ":" if "FT/IN" is present (case-insensitive)
  if "FT/IN" in depth.upper() or "FT/IN" in depth.lower():
        depth = depth.replace("'", ".")
        depth = depth.replace("ft/in", "ft./in.")
        depth = depth.replace("FT/IN", "ft./in.")


  # Replace "in" with "ft" if a period is present and "in" is present but "FTIN" or "FT:IN" are not
  if '.' in depth and 'in' in depth and 'FTIN' not in depth.upper() and 'FT:IN' not in depth.upper() and 'ft./in.' not in depth:
    depth = depth.replace('in', 'ft')

  # Remove the apostrophe if "0'" is present
  #if depth.startswith("0'"):
  #  depth = depth.replace("0'", '0')

  # Modifies the string by adding a " after the last number character if ' is found between two number characters.
  #depth = re.sub(r'(\d)\'(\d)', r'\1\'\2"', depth)  

  # Removes \ from string
  if '\\' in depth:
    depth = depth.replace('\\', '')
  
  # Removes extra double quotes from string
  if '""' in depth:
    depth = depth.replace('""', '"')


  return depth

## Loop thru a sheet and call Gemini

In [ ]:
import pandas as pd
import requests
from google.cloud import storage
from PIL import Image
import io
import json
import os

def read_sheet_call_gemini(csv_file):

    tmp_path='./img/tmp.jpeg'

    df = pd.read_csv(csv_file)

    df['ai_picture_quality'] = ''
    df['ai_quality_details'] = ''
    df['ai_easy_read'] = '' 
    df['ai_raw_depth'] = ''
    df['ai_normalized_depth'] = ''
    df['ai_reason'] = '' 

    for index, row in df.iterrows():
        url = row['URLPathText']
        golden_depth = row['Depth']

        try:
            

            raw_depth = ""
            print(f"Reading Image from {url} \n")

            #############

            #if os.path.exists(tmp_path):
            #    os.remove(tmp_path)
            #    print(f"File '{tmp_path}' deleted successfully.")
            #else:
            #    print(f"File '{tmp_path}' does not exist.")


            response=""
            # Create an empty DataFrame with 'depth' and 'full_response' columns
            response_df = pd.DataFrame(columns=['depth', 'full_response'])

            # Test the original image
            img = Image.open(requests.get(url, stream=True).raw)

            #response = requests.get(image_url, stream=False)  # Download the entire image at once
            #response.raise_for_status() # Raise an exception for bad status codes
            # Open the image using PIL
            #img = Image.open(response.content)  # Use response.content for the image data

            img.save(tmp_path,format="jpeg")
            img_bytes = img.tobytes()
            with open(tmp_path, "rb") as image2string: 
                base64_encoded_image = base64.b64encode(image2string.read()).decode('utf-8')  # Encode to base64 and decode to string 
            print("Original ...") 
            final_image = Part.from_data(mime_type="image/jpeg",data=base64.b64decode(base64_encoded_image))
            response = clean_response(generate(final_image))
            print ("Gemini response: " + response)

            ######
            # Parsing output
            data = json.loads(response)

            # Extract elements into variables
            raw_depth = data["raw_depth"]
            if raw_depth == "" or not raw_depth:
                raw_depth='Manual Review'
            else:
                raw_depth=raw_depth.replace(" ", "")

            # Example: Adding data to the DataFrame
            response_df.loc[len(response_df)] = [raw_depth, data]
            #######


            # Test the image with 90 degrees counter-clock rotation
            rotated_image1 = img.rotate(90)
            rotated_image1.save(tmp_path,format="jpeg")
            img_bytes = rotated_image1.tobytes()
            with open(tmp_path, "rb") as image2string: 
                base64_encoded_image = base64.b64encode(image2string.read()).decode('utf-8')  # Encode to base64 and decode to string 
            print("90 degrees ...") 
            final_image = Part.from_data(mime_type="image/jpeg",data=base64.b64decode(base64_encoded_image))
            response = clean_response(generate(final_image))
            print ("Gemini response: " + response)

            ######
            # Parsing output
            data = json.loads(response)

            # Extract elements into variables
            raw_depth = data["raw_depth"]
            if raw_depth == "" or not raw_depth:
                raw_depth='Manual Review'
            else:
                raw_depth=raw_depth.replace(" ", "")

            # Example: Adding data to the DataFrame
            response_df.loc[len(response_df)] = [raw_depth, data]
            #######


            # Test the image with 180 degrees counter-clock rotation
            rotated_image1 = img.rotate(180)
            rotated_image1.save(tmp_path,format="jpeg")
            img_bytes = rotated_image1.tobytes()
            with open(tmp_path, "rb") as image2string: 
                base64_encoded_image = base64.b64encode(image2string.read()).decode('utf-8')  # Encode to base64 and decode to string 
            print("180 degrees ...") 
            final_image = Part.from_data(mime_type="image/jpeg",data=base64.b64decode(base64_encoded_image))
            response = clean_response(generate(final_image))
            print ("Gemini response: " + response)

            ######
            # Parsing output
            data = json.loads(response)

            # Extract elements into variables
            raw_depth = data["raw_depth"]
            if raw_depth == "" or not raw_depth:
                raw_depth='Manual Review'
            else:
                raw_depth=raw_depth.replace(" ", "")

            # Example: Adding data to the DataFrame
            response_df.loc[len(response_df)] = [raw_depth, data]
            #######


            # Test the image with 275 degrees counter-clock rotation
            rotated_image1 = img.rotate(275)
            rotated_image1.save(tmp_path,format="jpeg")
            img_bytes = rotated_image1.tobytes()
            with open(tmp_path, "rb") as image2string: 
                base64_encoded_image = base64.b64encode(image2string.read()).decode('utf-8')  # Encode to base64 and decode to string 
            print("275 degrees ...") 
            final_image = Part.from_data(mime_type="image/jpeg",data=base64.b64decode(base64_encoded_image))
            response = clean_response(generate(final_image))
            print ("Gemini response: " + response)

            ######
            # Parsing output
            data = json.loads(response)

            # Extract elements into variables
            raw_depth = data["raw_depth"]
            if raw_depth == "" or not raw_depth:
                raw_depth='Manual Review'
            else:
                raw_depth=raw_depth.replace(" ", "")

            # Example: Adding data to the DataFrame
            response_df.loc[len(response_df)] = [raw_depth, data]
            #######


            # Test the image with 360 degrees counter-clock rotation
            rotated_image1 = img.rotate(360)
            rotated_image1.save(tmp_path,format="jpeg")
            img_bytes = rotated_image1.tobytes()
            with open(tmp_path, "rb") as image2string: 
                base64_encoded_image = base64.b64encode(image2string.read()).decode('utf-8')  # Encode to base64 and decode to string 
            print("360 degrees ...") 
            final_image = Part.from_data(mime_type="image/jpeg",data=base64.b64decode(base64_encoded_image))
            response = clean_response(generate(final_image))
            print ("Gemini response: " + response)

            ######
            # Parsing output
            data = json.loads(response)

            # Extract elements into variables
            raw_depth = data["raw_depth"]
            if raw_depth == "" or not raw_depth:
                raw_depth='Manual Review'
            else:
                raw_depth=raw_depth.replace(" ", "")

            # Example: Adding data to the DataFrame
            response_df.loc[len(response_df)] = [raw_depth, data]
            #######


            # Test the image with grayscale
            rotated_image1 = img.convert('L')
            rotated_image1.save(tmp_path,format="jpeg")
            img_bytes = rotated_image1.tobytes()
            with open(tmp_path, "rb") as image2string: 
                base64_encoded_image = base64.b64encode(image2string.read()).decode('utf-8')  # Encode to base64 and decode to string 
            print("Grayscale ...") 
            final_image = Part.from_data(mime_type="image/jpeg",data=base64.b64decode(base64_encoded_image))
            response = clean_response(generate(final_image))
            print ("Gemini response: " + response)
            ######
            # Parsing output
            data = json.loads(response)

            # Extract elements into variables
            raw_depth = data["raw_depth"]
            if raw_depth == "" or not raw_depth:
                raw_depth='Manual Review'
            else:
                raw_depth=raw_depth.replace(" ", "")

            # Example: Adding data to the DataFrame
            response_df.loc[len(response_df)] = [raw_depth, data]
            #######


            #print(response_df)
            most_frequent_row = get_most_frequent_depth(response_df)
            print("Depth:", most_frequent_row['depth'].iloc[0] if not most_frequent_row.empty else "No matching rows")
            print("Full Response:", most_frequent_row['full_response'].iloc[0] if not most_frequent_row.empty else "No matching rows")

            json_string = json.dumps(most_frequent_row['full_response'].iloc[0])
            data = json.loads(json_string)

            # post test image variations ...
            # # Extract elements into variables
            picture_quality = data["picture_quality"]
            quality_details = data["quality_details"]
            easy_read = data["easy_read"]
                        
            raw_depth = data["raw_depth"]
            if raw_depth == "" or not raw_depth:
                raw_depth='Manual Review'
            else:
                raw_depth=raw_depth.replace(" ", "")

            reason = data["reason"]

            # Print the extracted variables
            print(f"Picture Quality: {picture_quality}")
            print(f"Quality Details: {quality_details}")
            print(f"Easy to Read: {easy_read}")
            print(f"Raw Depth: {raw_depth}")
            print(f"Reason: {reason}")

            ############



            ### Append to the new dataframe    
            df.loc[index, 'ai_picture_quality'] = picture_quality
            df.loc[index, 'ai_quality_details'] = quality_details
            df.loc[index, 'ai_easy_read'] = easy_read
            df.loc[index, 'ai_raw_depth'] = raw_depth
            df.loc[index, 'ai_reason'] = reason
            df.loc[index, 'Depth'] = golden_depth.replace(" ", "")
            df.loc[index, 'ai_normalized_depth'] = modify_depth(raw_depth)

            print('\n')

        except Exception as e:
            print(f"Error processing from {url}: {e}")

    # Save DF to another sheet
    df.to_csv('output_data.csv', index=False)

    print("Done!")
    return df

# Example usage
csv_file = "/home/user/dycom_v1/dycom_sample_data_100.csv"
#csv_file = "./CQC_ED_SampleData_071624-SampleData.csv"

df=read_sheet_call_gemini(csv_file)

# Add to BQ
pandas_gbq.to_gbq(df, table_id, project_id=project_id, if_exists='replace')

In [ ]:
import datetime

now = datetime.datetime.now()

# Format the time as "HH:MM:SS"
formatted_time = now.strftime("%H:%M:%S")
print("Current time:", formatted_time)

Current time: 15:54:41


In [20]:
# Print the length of the DataFrame (number of rows)
print("Length of DataFrame:", len(df))

Length of DataFrame: 6


## Using Pandas_GBQ to create a new table output

In [ ]:
! pip install pandas-gbq

In [18]:
!bq mk dycom 

Dataset 'workstations-demo-420418:dycom' successfully created.


In [21]:
import pandas_gbq
# BQ Table as output
table_id='dycom.image_depth'
pandas_gbq.to_gbq(df, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 5548.02it/s]
